# Supervised Fine-tuning Trainer
Supervised fine-tuning (or SFT for short) is a crucial step in RLHF. In TRL we provide an easy-to-use API to create your SFT models and train them with few lines of code on your dataset.

Check out a complete flexible example at [`examples/scripts/sft.py`](https://github.com/huggingface/trl/tree/main/examples/scripts/sft.py). Experimental support for Vision Language Models is also included in the [`example examples/scripts/vsft_llava.py`](https://github.com/huggingface/trl/tree/main/examples/scripts/vsft_llava.py).
## Quickstart
If you have a dataset hosted on the Hub, you can easily fine-tune your SFT model using [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer) from TRL. Let us assume your dataset is imdb, the text you want to predict is inside the text field of the dataset, and you want to fine-tune the `facebook/opt-350m` model. The following code-snippet takes care of all the data pre-processing and training for you:

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer


dataset = load_dataset("imdb", split="train")

sft_config = SFTConfig(dataset_text_field="text", max_seq_length=512, output_dir="/tmp")
trainer = SFTTrainer("facebook/opt-350m", train_dataset=dataset, args=sft_config)
trainer.train()

Make sure to pass the correct value for `max_seq_length` as the default value will be set to `min`(`tokenizer.model_max_length`, `1024`).

You can also construct a model outside of the trainer and pass it as follows:

In [ ]:
from transformers import AutoModelForCausalLM

dataset = load_dataset("imdb", split="train")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

sft_config = SFTConfig(output_dir="/tmp")

trainer = SFTTrainer(model, train_dataset=dataset, args=sft_config)

trainer.train()

The above snippets will use the default training arguments from the [SFTConfig](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTConfig) class. If you want to modify the defaults pass in your modification to the `SFTConfig` constructor and pass them to the trainer via the args argument.

## Advanced usage
### Train on completions only
You can use the `DataCollatorForCompletionOnlyLM` to train your model on the generated prompts only. Note that this works only in the case when `packing=False`. To instantiate that collator for instruction data, pass a response template and the tokenizer. Here is an example of how it would work to fine-tune `opt-350m` on completions only on the CodeAlpaca dataset:

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
from trl import DataCollatorForCompletionOnlyLM


dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")


def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts


response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(model, train_dataset=dataset, args=SFTConfig(output_dir="/tmp"), formatting_func=formatting_prompts_func, data_collator=collator)

trainer.train()

To instantiate that collator for assistant style conversation data, pass a response template, an instruction template and the tokenizer. Here is an example of how it would work to fine-tune `opt-350m` on assistant completions only on the Open Assistant Guanaco dataset:

In [ ]:
dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

instruction_template = "### Human:"
response_template = "### Assistant:"
collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

trainer = SFTTrainer(model, args=SFTConfig(output_dir="/tmp"), train_dataset=dataset, data_collator=collator)

trainer.train()

Make sure to have a `pad_token_id` which is different from eos_token_id which can result in the model not properly predicting EOS (End of Sentence) tokens during generation.
### Using token_ids directly for response_template
Some tokenizers like Llama 2 (`meta-llama/Llama-2-XXb-hf`) tokenize sequences differently depending on whether they have context or not. For example:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")


def print_tokens_with_ids(txt):
    tokens = tokenizer.tokenize(txt, add_special_tokens=False)
    token_ids = tokenizer.encode(txt, add_special_tokens=False)
    print(list(zip(tokens, token_ids)))


prompt = """### User: Hello\n\n### Assistant: Hi, how can I help you?"""
print_tokens_with_ids(prompt)  # [..., ('▁Hello', 15043), ('<0x0A>', 13), ('<0x0A>', 13), ('##', 2277), ('#', 29937), ('▁Ass', 4007), ('istant', 22137), (':', 29901), ...]

response_template = "### Assistant:"
print_tokens_with_ids(response_template)  # [('▁###', 835), ('▁Ass', 4007), ('istant', 22137), (':', 29901)]

In this case, and due to lack of context in `response_template`, the same string (”### Assistant:”) is tokenized differently:

- Text (with context): [2277, 29937, 4007, 22137, 29901]
- `response_template` (without context): [835, 4007, 22137, 29901]

This will lead to an error when the DataCollatorForCompletionOnlyLM does not find the response_template in the dataset example text:

To solve this, you can tokenize the `response_template` with the same context as in the dataset, truncate it as needed and pass the `token_ids` directly to the `response_template` argument of the `DataCollatorForCompletionOnlyLM` class. For example:

In [ ]:
response_template_with_context = "\n### Assistant:"  # We added context here: "\n". This is enough for this tokenizer
response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[2:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`

data_collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

### Add Special Tokens for Chat Format
Adding special tokens to a language model is crucial for training chat models. These tokens are added between the different roles in a conversation, such as the user, assistant, and system and help the model recognize the structure and flow of a conversation. This setup is essential for enabling the model to generate coherent and contextually appropriate responses in a chat environment. The `setup_chat_format()` function in `trl` easily sets up a model and tokenizer for conversational AI tasks. This function:

- Adds special tokens to the tokenizer, e.g. `<|im_start|>` and `<|im_end|>`, to indicate the start and end of a conversation.
- Resizes the model’s embedding layer to accommodate the new tokens.
- Sets the `chat_template` of the tokenizer, which is used to format the input data into a chat-like format. The default is `chatml` from OpenAI.
- `optionally` you can pass `resize_to_multiple_of` to resize the embedding layer to a multiple of the `resize_to_multiple_of` argument, e.g. 64. If you want to see more formats being supported in the future, please open a GitHub issue on [trl](https://github.com/huggingface/trl)

In [ ]:
from trl import setup_chat_format


# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

# Set up the chat format with default 'chatml' format
model, tokenizer = setup_chat_format(model, tokenizer)

With our model and tokenizer set up, we can now fine-tune our model on a conversational dataset. Below is an example of how a dataset can be formatted for fine-tuning.

### Dataset format support
The [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer) supports popular dataset formats. This allows you to pass the dataset to the trainer without any pre-processing directly. The following formats are supported:

- conversational format

In [ ]:
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "..."}]}

- instruction format

In [ ]:
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

If your dataset uses one of the above formats, you can directly pass it to the trainer without pre-processing. The [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer) will then format the dataset for you using the defined format from the model’s tokenizer with the [`apply_chat_template`](https://huggingface.co/docs/transformers/main/en/chat_templating#templates-for-chat-models) method.

If the dataset is not in one of those format you can either preprocess the dataset to match the formatting or pass a formatting function to the SFTTrainer to do it for you.
### Customize your prompts using packed dataset
If your dataset has several fields that you want to combine, for example if the dataset has `question` and `answer` fields and you want to combine them, you can pass a formatting function to the trainer that will take care of that. For example:

In [ ]:
def formatting_func(example):
    text = f"### Question: {example['question']}\n ### Answer: {example['answer']}"
    return text


sft_config = SFTConfig(packing=True)
trainer = SFTTrainer("facebook/opt-350m", train_dataset=dataset, args=sft_config, formatting_func=formatting_func)

trainer.train()

You can also customize the `ConstantLengthDataset` much more by directly passing the arguments to the [`SFTConfig`](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTConfig) constructor. Please refer to that class’ signature for more information.
### Training adapters
We also support tight integration with PEFT library so that any user can conveniently train adapters and share them on the Hub instead of training the entire model

In [ ]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig


dataset = load_dataset("imdb", split="train")

peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

trainer = SFTTrainer("EleutherAI/gpt-neo-125m", train_dataset=dataset, args=SFTConfig(output_dir="/tmp"), peft_config=peft_config)

trainer.train()

You can also continue training your `PeftModel`. For that, first load a PeftModel outside `SFTTrainer` and pass it directly to the trainer without the `peft_config` argument being passed.
### Enhance the model’s performances using NEFTune
NEFTune is a technique to boost the performance of chat models and was introduced by the paper “[NEFTune: Noisy Embeddings Improve Instruction Finetuning](https://arxiv.org/abs/2310.05914)” from Jain et al. it consists of adding noise to the embedding vectors during training. According to the abstract of the paper:

"Standard finetuning of LLaMA-2-7B using Alpaca achieves 29.79% on AlpacaEval, which rises to 64.69% using noisy embeddings. NEFTune also improves over strong baselines on modern instruction datasets. Models trained with Evol-Instruct see a 10% improvement, with ShareGPT an 8% improvement, and with OpenPlatypus an 8% improvement. Even powerful models further refined with RLHF such as LLaMA-2-Chat benefit from additional training with NEFTune."

To use it in `SFTTrainer` simply pass `neftune_noise_alpha` when creating your `SFTConfig` instance. Note that to avoid any surprising behaviour, NEFTune is disabled after training to retrieve back the original behaviour of the embedding layer.

In [ ]:
sft_config = SFTConfig(neftune_noise_alpha=5,)
trainer = SFTTrainer("facebook/opt-350m", train_dataset=dataset, args=sft_config)
trainer.train()

We have tested NEFTune by training `mistralai/Mistral-7B-v0.1` on the [OpenAssistant](https://huggingface.co/datasets/timdettmers/openassistant-guanaco) dataset and validated that using NEFTune led to a performance boost of ~25% on MT Bench.

Note however, that the amount of performance gain is dataset dependent and in particular, applying NEFTune on synthetic datasets like [UltraChat](https://huggingface.co/datasets/stingning/ultrachat) typically produces smaller gains.
### Accelerate fine-tuning 2x using unsloth
You can further accelerate QLoRA / LoRA (2x faster, 60% less memory) using the unsloth library that is fully compatible with SFTTrainer. Currently unsloth supports only Llama (Yi, TinyLlama, Qwen, Deepseek etc) and Mistral architectures.

First install unsloth according to the official documentation. Once installed, you can incorporate unsloth into your workflow in a very simple manner; instead of loading AutoModelForCausalLM, you just need to load a FastLanguageModel as follows:

In [ ]:
from unsloth import FastLanguageModel


max_seq_length = 2048 # Supports automatic RoPE Scaling, so choose any number

# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b",
    max_seq_length=max_seq_length,
    dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  # Dropout = 0 is currently optimized
    bias="none",  # Bias = "none" is currently optimized
    use_gradient_checkpointing=True,
    random_state=3407,
)

args = SFTConfig(output_dir="./output", max_seq_length=max_seq_length, dataset_text_field="text")

trainer = SFTTrainer(model=model, args=args, train_dataset=dataset)
trainer.train()

The saved model is fully compatible with Hugging Face’s transformers library. Learn more about unsloth in their [official repository](https://github.com/unslothai/unsloth).
## Best practices
Pay attention to the following best practices when training a model with that trainer:
- [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer) always pads by default the sequences to the `max_seq_length` argument of the [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer). If none is passed, the trainer will retrieve that value from the tokenizer. Some tokenizers do not provide a default value, so there is a check to retrieve the minimum between 2048 and that value. Make sure to check it before training.
- For training adapters in 8bit, you might need to tweak the arguments of the `prepare_model_for_kbit_training` method from PEFT, hence we advise users to use `prepare_in_int8_kwargs` field, or create the PeftModel outside the [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer) and pass it.
- For a more memory-efficient training using adapters, you can load the base model in 8bit, for that simply add `load_in_8bit` argument when creating the [SFTTrainer](https://huggingface.co/docs/trl/v0.9.4/en/sft_trainer#trl.SFTTrainer), or create a base model in 8bit outside the trainer and pass it.
- If you create a model outside the trainer, make sure to not pass to the trainer any additional keyword arguments that are relative to `from_pretrained()` method.
## Multi-GPU Training
Trainer (and thus SFTTrainer) supports multi-GPU training. If you run your script with python script.py it will default to using DP as the strategy, which may be [slower than expected](https://github.com/huggingface/trl/issues/1303). To use DDP (which is generally recommended, see [here](https://huggingface.co/docs/transformers/en/perf_train_gpu_many?select-gpu=Accelerate#data-parallelism) for more info) you must launch the script with `python -m torch.distributed.launch script.py` or `accelerate launch script.py`. For DDP to work you must also check the following:
- If you’re using gradient_checkpointing, add the following to the TrainingArguments: `gradient_checkpointing_kwargs={'use_reentrant':False}` (more info here
- Ensure that the model is placed on the correct device
## GPTQ Conversion
You may experience some issues with GPTQ Quantization after completing training. Lowering `gradient_accumulation_steps` to 4 will resolve most issues during the quantization process to GPTQ format.